In [ ]:
#Set up paths
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime
import yaml
import glob
#from box import Box #???

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(GrU)


##########################################################
# Read YAML file and open xarray Datasets
#---------------------------------------------------------

with open('configure_ana_plots.yaml', 'r') as file:
    cfg = yaml.safe_load(file)


x='ne240x2_QxQsst_smth'
x='ne240x2'

exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
ymdPat = "2004-06-01_2004-06-30" #cfg[x]['ymdPat']

Bdiro = f"/glade/derecho/scratch/juliob/archive/{exp}/atm/GWana/"

fname=f"{exp}_GWana_xPyP_{ymdPat}.nc"
print(Bdiro+fname )
XX = xr.open_mfdataset( Bdiro+fname )
fname=f"{exp}_GWana_zeta_{ymdPat}.nc"
print(Bdiro+fname )
Xzt = xr.open_mfdataset( Bdiro+fname )
fname=f"{exp}_GWana_PBLq_{ymdPat}.nc"
print(Bdiro+fname )
Xpbl = xr.open_mfdataset( Bdiro+fname )






clat2,clon2 = GrU.latlon( grid='ne16pg3' )
lat1,lon1 = GrU.latlon( grid='fv1x1' )

days = np.arange( len(XX.time.values) )/4.



In [ ]:
plt.contourf( np.sqrt(Xzt.zeta2[10,1,:,:]) , cmap='plasma' )
plt.colorbar()


In [ ]:
devUW_x1 = XX.uPomegaP.values
print( np.shape( devUW_x1 ) )
nt,nz,ny,nx = np.shape( devUW_x1 )


zeta_x1=Xzt.zeta.values
zeta2_x1=Xzt.zeta2.values


In [ ]:
#################################
# weird stuff for axes

from matplotlib.ticker import FuncFormatter

def plot_with_formatter(ax, xtick_positions_m, m_per_deg, ytick_positions_s):

    def m_to_lon(x):
        return (x / m_per_deg) #- 180.

    def longitude_formatter(x, pos):
        degree_symbol = '\u00B0'
        return f'{m_to_lon(x):.1f}{degree_symbol}'

    def s_to_day(y):
        return (y / 86400. ) #- 180.

    def day_formatter(y, pos):
        return f'{s_to_day(y):.1f}'

    ax.set_xticks(xtick_positions_m)
    ax.xaxis.set_major_formatter(FuncFormatter(longitude_formatter))
    ax.set_yticks(ytick_positions_s)
    ax.yaxis.set_major_formatter(FuncFormatter(day_formatter))

jj = 40 #45 #40

latval = lat1[jj]


circum = 2.*np.pi*6_371_000* np.cos( np.radians( latval ) ) # circumfrerence along lat in m
if latval < 0:
    ILat=f'{str(int(-latval))}S'
else:
    ILat=f'{str(int(latval))}N'

m_per_deg = circum / 360. 

# Define tick positions in degrees and convert to km
#tick_positions_deg = [60, 120, 180, 240, 300]
xtick_positions_deg = [-120, -60, 0, 60, 120]
xtick_positions_m = [(deg+180.)* m_per_deg for deg in xtick_positions_deg]  # Assuming 68 km per degree
ytick_positions_day =[1,10,20,30]
ytick_positions_s = [day * 86400.  for day in ytick_positions_day]

##########################################################
# Make fancy plots
#---------------------------------------------------------

fig, ax = plt.subplots(2, 3, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

dolev = np.linspace(-1,1,num=41)
olev = np.linspace(-10,10,num=21)


tootrange = f" {(XX.time[0].values.item() ).strftime('%Y-%m-%d')} to {(XX.time[-1].values.item() ).strftime('%Y-%m-%d')}"
dayrange=f"{(XX.time[0].values.item() ).strftime('%Y-%m-%d')}-{(XX.time[-1].values.item() ).strftime('%d')}"

xx=np.linspace(0,1.,num=nx)*circum     #( 2.*np.pi*6_371_000* np.cos( np.radians( latval ) ))
tt=np.linspace(0,1.,num=nt)*6.*3600.*nt


print(f" Lat={latval:.1f} ")
print(tootrange )

plev=XX.lev.values
zlev=-7.0*np.log(plev/1_000.)
klist=np.arange( len(zlev) )
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    if zlev[k]>30. :
        scak=scak*0.1

    #co=ax[p].contourf(xx, tt, devUW_x1[:,k,jj,:],levels=2*dolev/scak ,cmap='bwr' )
    #col=ax[p].contourf(xx, tt, zeta_x1[:,0,jj,:],levels=3 , cmap='PiYG' , alpha=.5) #, colors='black' , alpha=.2)
    co=ax[p].contourf(xx, tt, np.average( devUW_x1[:,k,jj-5:jj+5,:] ,axis=1 ),levels=1*dolev/scak ,cmap='bwr' )
    col=ax[p].contourf(xx, tt, np.average( zeta_x1[:,2,jj-5:jj+5,:] ,axis=1 ),levels=3 , cmap='PiYG' , alpha=.3) #, colors='black' , alpha=.2)
    ln0=ax[p].plot( tt*5 , tt, color='black'  ,  label='5ms$^{-1}$'   )
    ln0=ax[p].plot( tt*10 , tt, '--', color='black' ,  label='10ms$^{-1}$' )
    ln0=ax[p].plot( tt*20 , tt, '-.', color='black' ,  label='20ms$^{-1}$' )
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    ax[p].set_ylabel( 'Days' )
    ax[p].set_xlabel( 'Longitude' )
    ax[p].set_xlim( 0,xx.max())
    
    # Set the tick positions and formatter for the x-axis
    #ax[p].set_xticks(tick_positions_m)
    #ax[p].xaxis.set_major_formatter(FuncFormatter(longitude_formatter(tick_positions_m,m_per_deg,))
    plot_with_formatter(ax[p], xtick_positions_m, m_per_deg,ytick_positions_s)
    ax[p].legend()
    cb=plt.colorbar( co )
    p=p+1

fig.suptitle( f'{exp}, {tootrange}, Lat={latval:.1f}' , fontsize=20)

FigName=f'Hpass-devUW-xtHovmueller_y{ILat}_{dayrange}_{exp}.png'

print(FigName)
Pu.save_link(png_filename = FigName )

In [ ]:
###
print( zlev)


In [ ]:
#################################
# weird stuff for axes
from scipy.ndimage import gaussian_filter as GF
from matplotlib.ticker import FuncFormatter

def plot_with_formatter(ax, xtick_positions_m, m_per_deg, ytick_positions_s):

    def m_to_lon(x):
        return (x / m_per_deg) #- 180.

    def longitude_formatter(x, pos):
        degree_symbol = '\u00B0'
        return f'{m_to_lon(x):.1f}{degree_symbol}'

    def s_to_day(y):
        return (y / 86400. )+1. #- 180.

    def day_formatter(y, pos):
        return f'{s_to_day(y):.1f}'

    ax.set_xticks(xtick_positions_m)
    ax.xaxis.set_major_formatter(FuncFormatter(longitude_formatter))
    ax.set_yticks(ytick_positions_s)
    ax.yaxis.set_major_formatter(FuncFormatter(day_formatter))

lat_sel= uti.find_nearest_plev_indices(plev=lat1  , target_levels=[-41.])

jj = lat_sel[0] # 45 #45 #40


latval = lat1[jj]

circum = 2.*np.pi*6_371_000* np.cos( np.radians( latval ) ) # circumfrerence along lat in m
if latval < 0:
    ILat=f'{str(int(-latval))}S'
else:
    ILat=f'{str(int(latval))}N'

m_per_deg = circum / 360. 

# Define tick positions in degrees and convert to km
#tick_positions_deg = [60, 120, 180, 240, 300]
xtick_positions_deg = [-120, -60, 0, 60, 120]
xtick_positions_m = [(deg+180.)* m_per_deg for deg in xtick_positions_deg]  # Assuming 68 km per degree
ytick_positions_day =[1,10,20,30]
ytick_positions_s = [(day-1) * 86400.  for day in ytick_positions_day]

##########################################################
# Make fancy plots
#---------------------------------------------------------

fig, ax = plt.subplots(1, 3, figsize=(28, 7))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

dolev = np.linspace(-1,1,num=41)
olev = np.linspace(-10,10,num=21)
vorlev=1.* np.linspace( 0, 0.0005, num=31 )

tootrange = f" {(XX.time[0].values.item() ).strftime('%Y-%m-%d')} to {(XX.time[-1].values.item() ).strftime('%Y-%m-%d')}"
dayrange=f"{(XX.time[0].values.item() ).strftime('%Y-%m-%d')}-{(XX.time[-1].values.item() ).strftime('%d')}"

xx=np.linspace(0,1.,num=nx)*circum     #( 2.*np.pi*6_371_000* np.cos( np.radians( latval ) ))
tt=np.linspace(0,1.,num=nt)*6.*3600.*nt 


print(f" Lat={latval:.1f} ")
print(tootrange )

plev=XX.lev.values
zlev=-7.0*np.log(plev/1_000.)
klist=np.arange( len(zlev) )
p=0
k=4
k_vort=1
print( plev[k] , zlev[k] )
scak = 500./ plev[k]
if zlev[k]>30. :
    scak=scak*0.1

tit1=r"$<\zeta^2>$"

#co=ax[p].contourf(xx, tt, devUW_x1[:,k,jj,:],levels=2*dolev/scak ,cmap='bwr' )
#col=ax[p].contourf(xx, tt, zeta_x1[:,0,jj,:],levels=3 , cmap='PiYG' , alpha=.5) #, colors='black' , alpha=.2)
#co=ax[p].contourf(xx, tt, np.average( devUW_x1[:,k,jj-5:jj+5,:] ,axis=1 ),levels=1*dolev/scak ,cmap='bwr' )
col=ax[p].contourf(xx, tt, np.sqrt( np.average( zeta2_x1[:,k_vort,jj-5:jj+5,:] ,axis=1 ) ),levels=vorlev , cmap='plasma' , alpha=.5) #, colors='black' , alpha=.2)
ln0=ax[p].plot( tt*5 , tt, color='black'  ,  label='5ms$^{-1}$'   )
ln0=ax[p].plot( tt*10 , tt, '--', color='black' ,  label='10ms$^{-1}$' )
ln0=ax[p].plot( tt*20 , tt, '-.', color='black' ,  label='20ms$^{-1}$' )
ax[p].set_title( f'{tit1} {plev[k_vort]:.1f} hPa, {zlev[k_vort]:.1f} km' )
ax[p].set_ylabel( 'Days' )
ax[p].set_xlabel( 'Longitude' )
ax[p].set_xlim( 0,xx.max())

# Set the tick positions and formatter for the x-axis
#ax[p].set_xticks(tick_positions_m)
#ax[p].xaxis.set_major_formatter(FuncFormatter(longitude_formatter(tick_positions_m,m_per_deg,))
plot_with_formatter(ax[p], xtick_positions_m, m_per_deg,ytick_positions_s)
ax[p].legend()
ax[p].text(-0.08, 1.05, f"{chr(97 + p )})", transform=ax[p].transAxes,
    fontsize=16, fontweight='bold', va='top')
cb=plt.colorbar( col )

p=p+1
tit1=r"$\overline{u'\omega'}$"
#co=ax[p].contourf(xx, tt, devUW_x1[:,k,jj,:],levels=2*dolev/scak ,cmap='bwr' )
#col=ax[p].contourf(xx, tt, zeta_x1[:,0,jj,:],levels=3 , cmap='PiYG' , alpha=.5) #, colors='black' , alpha=.2)
co=ax[p].contourf(xx, tt, np.average( devUW_x1[:,k,jj-5:jj+5,:] ,axis=1 ),levels=1*dolev/scak ,cmap='bwr' )
#col=ax[p].contourf(xx, tt, np.average( zeta_x1[:,2,jj-5:jj+5,:] ,axis=1 ),levels=11 , cmap='PiYG' , alpha=.3) #, colors='black' , alpha=.2)
ln0=ax[p].plot( tt*5 , tt, color='black'  ,  label='5ms$^{-1}$'   )
ln0=ax[p].plot( tt*10 , tt, '--', color='black' ,  label='10ms$^{-1}$' )
ln0=ax[p].plot( tt*20 , tt, '-.', color='black' ,  label='20ms$^{-1}$' )
ax[p].set_title( f'{tit1} {plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
ax[p].set_ylabel( 'Days' )
ax[p].set_xlabel( 'Longitude' )
ax[p].set_xlim( 0,xx.max())

# Set the tick positions and formatter for the x-axis
#ax[p].set_xticks(tick_positions_m)
#ax[p].xaxis.set_major_formatter(FuncFormatter(longitude_formatter(tick_positions_m,m_per_deg,))
plot_with_formatter(ax[p], xtick_positions_m, m_per_deg,ytick_positions_s)
ax[p].legend()
ax[p].text(-0.08, 1.05, f"{chr(97 + p )})", transform=ax[p].transAxes,
    fontsize=16, fontweight='bold', va='top')

cb=plt.colorbar( co )

p=p+1
#co=ax[p].contourf(xx, tt, devUW_x1[:,k,jj,:],levels=2*dolev/scak ,cmap='bwr' )
#col=ax[p].contourf(xx, tt, zeta_x1[:,0,jj,:],levels=3 , cmap='PiYG' , alpha=.5) #, colors='black' , alpha=.2)
co=ax[p].contourf(xx, tt, np.average( devUW_x1[:,k,jj-5:jj+5,:] ,axis=1 ),levels=1*dolev/scak ,cmap='bwr' )
#col=ax[p].contourf(xx, tt, np.sqrt( np.average( zeta2_x1[:,k_vort,jj-5:jj+5,:] ,axis=1 ) ),levels=vorlev , cmap='plasma' , alpha=.1) #, colors='black' , alpha=.2)
col=ax[p].contour(xx, tt, GF( np.sqrt( np.average( zeta2_x1[:,k_vort,jj-5:jj+5,:] ,axis=1 ) ),sigma=[0,1]),levels=vorlev[[5,30]] , colors='black', alpha=.5) #, colors='black' , alpha=.2)
ln0=ax[p].plot( tt*5 , tt, color='black'  ,  label='5ms$^{-1}$'   )
ln0=ax[p].plot( tt*10 , tt, '--', color='black' ,  label='10ms$^{-1}$' )
ln0=ax[p].plot( tt*20 , tt, '-.', color='black' ,  label='20ms$^{-1}$' )
ax[p].set_title( f'Overlay' )
ax[p].set_ylabel( 'Days' )
ax[p].set_xlabel( 'Longitude' )
ax[p].set_xlim( 0,xx.max())

# Set the tick positions and formatter for the x-axis
#ax[p].set_xticks(tick_positions_m)
#ax[p].xaxis.set_major_formatter(FuncFormatter(longitude_formatter(tick_positions_m,m_per_deg,))
plot_with_formatter(ax[p], xtick_positions_m, m_per_deg,ytick_positions_s)
ax[p].legend()
ax[p].text(-0.08, 1.05, f"{chr(97 + p )})", transform=ax[p].transAxes,
    fontsize=16, fontweight='bold', va='top')

cb=plt.colorbar( co )

fig.suptitle( f'{exp}, {tootrange}, Lat=<{latval-5:.0f},{latval+5:.0f}>' , fontsize=20)

FigName=f'Hpass-devUW-xtHovmueller_y{ILat}_{dayrange}_{exp}.png'

print(FigName)
Pu.save_link(png_filename = FigName )

In [ ]:

print(" STOPPPPPP " )


In [ ]:

print(" STOPPPPPP " )


In [ ]:

print(" STOPPPPPP " )


In [ ]:
wp2_x1=Xpbl.wp2.values

In [ ]:
#################################
# weird stuff for axes
from scipy.ndimage import gaussian_filter as GF
from matplotlib.ticker import FuncFormatter

def plot_with_formatter(ax, xtick_positions_m, m_per_deg, ytick_positions_s):

    def m_to_lon(x):
        return (x / m_per_deg) #- 180.

    def longitude_formatter(x, pos):
        degree_symbol = '\u00B0'
        return f'{m_to_lon(x):.1f}{degree_symbol}'

    def s_to_day(y):
        return (y / 86400. ) #- 180.

    def day_formatter(y, pos):
        return f'{s_to_day(y):.1f}'

    ax.set_xticks(xtick_positions_m)
    ax.xaxis.set_major_formatter(FuncFormatter(longitude_formatter))
    ax.set_yticks(ytick_positions_s)
    ax.yaxis.set_major_formatter(FuncFormatter(day_formatter))

jj = 45 #45 #40



circum = 2.*np.pi*6_371_000* np.cos( np.radians( latval ) ) # circumfrerence along lat in m
latval = lat1[jj]
if latval < 0:
    ILat=f'{str(int(-latval))}S'
else:
    ILat=f'{str(int(latval))}N'

m_per_deg = circum / 360. 

# Define tick positions in degrees and convert to km
#tick_positions_deg = [60, 120, 180, 240, 300]
xtick_positions_deg = [-120, -60, 0, 60, 120]
xtick_positions_m = [(deg+180.)* m_per_deg for deg in xtick_positions_deg]  # Assuming 68 km per degree
ytick_positions_day =[1,10,20,30]
ytick_positions_s = [day * 86400.  for day in ytick_positions_day]

##########################################################
# Make fancy plots
#---------------------------------------------------------

fig, ax = plt.subplots(1, 3, figsize=(28, 7))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

dolev = np.linspace(-1,1,num=41)
olev = np.linspace(-10,10,num=21)
vorlev=np.linspace( 0, 0.0005, num=31 )

tootrange = f" {(XX.time[0].values.item() ).strftime('%Y-%m-%d')} to {(XX.time[-1].values.item() ).strftime('%Y-%m-%d')}"
dayrange=f"{(XX.time[0].values.item() ).strftime('%Y-%m-%d')}-{(XX.time[-1].values.item() ).strftime('%d')}"

xx=np.linspace(0,1.,num=nx)*circum     #( 2.*np.pi*6_371_000* np.cos( np.radians( latval ) ))
tt=np.linspace(0,1.,num=nt)*6.*3600.*nt


print(f" Lat={latval:.1f} ")
print(tootrange )

plev=XX.lev.values
zlev=-7.0*np.log(plev/1_000.)
plev_pbl=Xpbl.lev.values
zlev_pbl=-7.0*np.log(plev_pbl/1_000.)
klist=np.arange( len(zlev) )
p=0
k=4
k_pbl=0
print( plev[k] , zlev[k] )
scak = 500./ plev[k]
if zlev[k]>30. :
    scak=scak*0.1

#co=ax[p].contourf(xx, tt, devUW_x1[:,k,jj,:],levels=2*dolev/scak ,cmap='bwr' )
#col=ax[p].contourf(xx, tt, zeta_x1[:,0,jj,:],levels=3 , cmap='PiYG' , alpha=.5) #, colors='black' , alpha=.2)
#co=ax[p].contourf(xx, tt, np.average( devUW_x1[:,k,jj-5:jj+5,:] ,axis=1 ),levels=1*dolev/scak ,cmap='bwr' )
col=ax[p].contourf(xx, tt, np.sqrt( np.average( wp2_x1[:,k_pbl,jj-5:jj+5,:] ,axis=1 ) ),levels=31 , cmap='plasma' , alpha=.5) #, colors='black' , alpha=.2)
ln0=ax[p].plot( tt*5 , tt, color='black'  ,  label='5ms$^{-1}$'   )
ln0=ax[p].plot( tt*10 , tt, '--', color='black' ,  label='10ms$^{-1}$' )
ln0=ax[p].plot( tt*20 , tt, '-.', color='black' ,  label='20ms$^{-1}$' )
ax[p].set_title( f'{plev_pbl[k_pbl]:.1f} hPa, {zlev_pbl[k_vort]:.1f} km' )
ax[p].set_ylabel( 'Days' )
ax[p].set_xlabel( 'Longitude' )
ax[p].set_xlim( 0,xx.max())

# Set the tick positions and formatter for the x-axis
#ax[p].set_xticks(tick_positions_m)
#ax[p].xaxis.set_major_formatter(FuncFormatter(longitude_formatter(tick_positions_m,m_per_deg,))
plot_with_formatter(ax[p], xtick_positions_m, m_per_deg,ytick_positions_s)
ax[p].legend()
ax[p].text(-0.08, 1.05, f"{chr(97 + p )})", transform=ax[p].transAxes,
    fontsize=16, fontweight='bold', va='top')
cb=plt.colorbar( col )

p=p+1
#co=ax[p].contourf(xx, tt, devUW_x1[:,k,jj,:],levels=2*dolev/scak ,cmap='bwr' )
#col=ax[p].contourf(xx, tt, zeta_x1[:,0,jj,:],levels=3 , cmap='PiYG' , alpha=.5) #, colors='black' , alpha=.2)
co=ax[p].contourf(xx, tt, np.average( devUW_x1[:,k,jj-5:jj+5,:] ,axis=1 ),levels=1*dolev/scak ,cmap='bwr' )
#col=ax[p].contourf(xx, tt, np.average( zeta_x1[:,2,jj-5:jj+5,:] ,axis=1 ),levels=11 , cmap='PiYG' , alpha=.3) #, colors='black' , alpha=.2)
ln0=ax[p].plot( tt*5 , tt, color='black'  ,  label='5ms$^{-1}$'   )
ln0=ax[p].plot( tt*10 , tt, '--', color='black' ,  label='10ms$^{-1}$' )
ln0=ax[p].plot( tt*20 , tt, '-.', color='black' ,  label='20ms$^{-1}$' )
ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
ax[p].set_ylabel( 'Days' )
ax[p].set_xlabel( 'Longitude' )
ax[p].set_xlim( 0,xx.max())

# Set the tick positions and formatter for the x-axis
#ax[p].set_xticks(tick_positions_m)
#ax[p].xaxis.set_major_formatter(FuncFormatter(longitude_formatter(tick_positions_m,m_per_deg,))
plot_with_formatter(ax[p], xtick_positions_m, m_per_deg,ytick_positions_s)
ax[p].legend()
ax[p].text(-0.08, 1.05, f"{chr(97 + p )})", transform=ax[p].transAxes,
    fontsize=16, fontweight='bold', va='top')

cb=plt.colorbar( co )

p=p+1
#co=ax[p].contourf(xx, tt, devUW_x1[:,k,jj,:],levels=2*dolev/scak ,cmap='bwr' )
#col=ax[p].contourf(xx, tt, zeta_x1[:,0,jj,:],levels=3 , cmap='PiYG' , alpha=.5) #, colors='black' , alpha=.2)
co=ax[p].contourf(xx, tt, np.average( devUW_x1[:,k,jj-5:jj+5,:] ,axis=1 ),levels=1*dolev/scak ,cmap='bwr' )
#col=ax[p].contourf(xx, tt, np.sqrt( np.average( wp2_x1[:,k_pbl,jj-5:jj+5,:] ,axis=1 ) ),levels=31 , cmap='plasma' , alpha=.5) #, colors='black' , alpha=.2)
col=ax[p].contour(xx, tt, GF( np.sqrt( np.average( wp2_x1[:,k_vort,jj-5:jj+5,:] ,axis=1 ) ),sigma=[0,1]),levels=2 , colors='black', alpha=.5) #, colors='black' , alpha=.2)
ln0=ax[p].plot( tt*5 , tt, color='black'  ,  label='5ms$^{-1}$'   )
ln0=ax[p].plot( tt*10 , tt, '--', color='black' ,  label='10ms$^{-1}$' )
ln0=ax[p].plot( tt*20 , tt, '-.', color='black' ,  label='20ms$^{-1}$' )
ax[p].set_title( f'Overlay' )
ax[p].set_ylabel( 'Days' )
ax[p].set_xlabel( 'Longitude' )
ax[p].set_xlim( 0,xx.max())

# Set the tick positions and formatter for the x-axis
#ax[p].set_xticks(tick_positions_m)
#ax[p].xaxis.set_major_formatter(FuncFormatter(longitude_formatter(tick_positions_m,m_per_deg,))
plot_with_formatter(ax[p], xtick_positions_m, m_per_deg,ytick_positions_s)
ax[p].legend()
ax[p].text(-0.08, 1.05, f"{chr(97 + p )})", transform=ax[p].transAxes,
    fontsize=16, fontweight='bold', va='top')

cb=plt.colorbar( co )

fig.suptitle( f'{exp}, {tootrange}, Lat=<{latval-5:.0f},{latval+5:.0f}>' , fontsize=20)

FigName=f'Hpass-devUW-xtHovmueller_y{ILat}_{dayrange}_{exp}.png'

print(FigName)
Pu.save_link(png_filename = FigName )

In [ ]:
print(jj)
sgh=Xpbl.sgh.values


In [ ]:
no_oro=np.where( sgh<20, 1, 0)

In [ ]:
plt.contour( no_oro )
plt.colorbar()

In [ ]:
print( devUW_x1.shape )


In [ ]:
for t in np.arange(119):
    for z in np.arange(6):
        devUW_no_oro[t,z,:,:]=no_oro*devUW_x1[t,z,:,:]

In [ ]:
plt.scatter( zeta2_x1[:,1,40:50,144],devUW_x1[:,4,40:50,144] )
#plt.ylim( (-0.1,0.1))

In [ ]:
meanUW_x1 = np.average( devUW_x1, axis=0)  
uwlevs = np.linspace(-.2,.2,41) 
#mvarTS_x1 = np.average(np.sqrt(varTS_x1),axis=0)

fig, ax = plt.subplots(2, 3, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()



tootrange = f" {(XX.time[0].values.item() ).strftime('%Y-%m-%d')} to {(XX.time[-1].values.item() ).strftime('%Y-%m-%d')}"
dayrange=f"{(XX.time[0].values.item() ).strftime('%Y-%m-%d')}-{(XX.time[-1].values.item() ).strftime('%d')}"

plev=XX.lev.values
zlev=-7.0*np.log(plev/1_000.)
klist=np.arange( len(zlev) )
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 0.5*500./ plev[k]
    if zlev[k]>30. :
        scak=scak*0.05
    #co=ax[p].contourf( scak*ogOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co=ax[p].contourf( lon1, lat1, meanUW_x1[k,:,:] ,levels=(1./scak)*uwlevs, cmap='bwr' )
    #cot=ax[p].contour( lon1, lat1 , mvarTS_x1, levels=[ .1,.2,.3,.5, 1., 2.] , colors='black', alpha=.5 )
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    ax[p].set_ylabel( 'Latitude' )
    ax[p].set_xlabel( 'Longitude' )
    cb=plt.colorbar( co )
    p=p+1

fig.suptitle( f'{exp}, {tootrange}', fontsize=20)

FigName=f'Hpass-devUW-xyMap_{dayrange}_{exp}.png'

print(FigName)
Pu.save_link(png_filename = FigName )


In [ ]:
# Save results
import copy
if x=='ne240x2_QxQsst':
    print( x )
    meanUW_x1_QxQ = copy.deepcopy( meanUW_x1 )
elif x=='ne240x2':
    print( x )
    meanUW_x1_ctl = copy.deepcopy( meanUW_x1 )
elif x=='ne240x2_L56':
    print( x )
    meanUW_x1_L56 = copy.deepcopy( meanUW_x1 )
else:
    print( "No vakues of exp " )



In [ ]:
dmeanUW_x1 = meanUW_x1_QxQ - meanUW_x1_ctl  #np.average( devUW_x1, axis=0)  
eet='Diff-meanUW_x1_QxQ-meanUW_x1_ctl'
uwlevs = 0.5*np.linspace(-.2,.2,41) 
mvarTS_x1 = np.average(np.sqrt(varTS_x1),axis=0)

fig, ax = plt.subplots(2, 3, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()



tootrange = f" {(XX.time[0].values.item() ).strftime('%Y-%m-%d')} to {(XX.time[-1].values.item() ).strftime('%Y-%m-%d')}"
dayrange=f"{(XX.time[0].values.item() ).strftime('%Y-%m-%d')}-{(XX.time[-1].values.item() ).strftime('%d')}"

plev=XX.lev.values
zlev=-7.0*np.log(plev/1_000.)
klist=np.arange( len(zlev) )
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 0.5*500./ plev[k]
    if zlev[k]>30. :
        scak=scak*0.05
    #co=ax[p].contourf( scak*ogOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co=ax[p].contourf( lon1, lat1, dmeanUW_x1[k,:,:] ,levels=(1./scak)*uwlevs, cmap='bwr', alpha=0.5 )
    #cot=ax[p].contour( lon1, lat1 , mvarTS_x1, levels=[ .1,.3,.5,.7,.9] , cmap='plasma', alpha=.15 )
    cot=ax[p].contourf( lon1, lat1 , mvarTS_x1, levels=[ .1,.3,.5,.7,.9] , cmap='Greens', alpha=.2 )
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    ax[p].set_ylabel( 'Latitude' )
    ax[p].set_xlabel( 'Longitude' )
    cb=plt.colorbar( co )
    p=p+1

fig.suptitle( f'{eet}, {tootrange}', fontsize=20)

FigName=f'Hpass-devUW-xyMap_w_SST_{dayrange}_{eet}.png'

print(FigName)
Pu.save_link(png_filename = FigName )


In [ ]:
tsO=XTS.tsO.values

tsO_XX=RgF.Horz(xfld_Src=tsO , Src='ne240pg3', Dst='latlonOXX' , RegridObj_In=  RgOb_ne240_x_llOXX) 
devTS_XX=RgF.Horz(xfld_Src=devTS , Src='ne240pg3', Dst='latlonOXX' , RegridObj_In=  RgOb_ne240_x_llOXX) 


latO,lonO = GrU.latlon( grid='latlonOXX' )

In [ ]:
print(latO[301])

In [ ]:
plt.contourf( lonO, days, devTS_XX[:,301,:] , levels=np.linspace(-1,1,41))

In [ ]:
plt.contourf( lon1, days, varTS_x1[:,40,:] , levels=np.linspace(-1,1,41))

In [ ]:
plt.contour( lon1, lat1 , np.average(np.sqrt(varTS_x1),axis=0) , levels=[ .1,.2,.3,.5, 1., 2.] , colors='black' )

In [ ]:
##########################################################
# Make fancy plots
#---------------------------------------------------------

fig, ax = plt.subplots(2, 3, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

dolev = np.linspace(-1,1,num=41)
olev = np.linspace(-10,10,num=21)

jj = 40


tootrange = f" {(XX.time[0].values.item() ).strftime('%Y-%m-%d')} to {(XX.time[-1].values.item() ).strftime('%Y-%m-%d')}"
dayrange=f"{(XX.time[0].values.item() ).strftime('%Y-%m-%d')}-{(XX.time[-1].values.item() ).strftime('%d')}"
#loot=f"Lat={A.X.lat[jj].values:.1f}"
latval = lat1[jj]
if latval < 0:
    ILat=f'{str(int(-latval))}S'
else:
    ILat=f'{str(int(latval))}N'

print(f" Lat={latval:.1f} ")
print(tootrange )

plev=XX.lev.values
zlev=-7.0*np.log(plev/1_000.)
klist=np.arange( len(zlev) )
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    if zlev[k]>30. :
        scak=scak*0.25
    #co=ax[p].contourf( scak*ogOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co=ax[p].contourf(lon1, days, scak* devUW_x1[:,k,jj,:],levels=2*dolev ,cmap='bwr' )

    #cots = ax[p].contourf(lonO, days, devTS_XX[:,301,:],levels=5*dolev , cmap='bwr' , alpha=0.5 )
    cots = ax[p].contourf(lon1, days, varTS_x1[:,jj,:],levels=5*dolev , cmap='gist_ncar' , alpha=0.25 )
    
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    ax[p].set_ylabel( 'Days' )
    ax[p].set_xlabel( 'Longitude' )
    cb=plt.colorbar( co )
    p=p+1

fig.suptitle( f'{exp}, {tootrange}, Lat={latval:.1f}' , fontsize=20)


In [ ]:
##########################################################
# Make fancy plots
#---------------------------------------------------------

fig, ax = plt.subplots(2, 3, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

dolev = 5*np.linspace(0,1,num=41)
olev = np.linspace(-10,10,num=21)

jj = 30


tootrange = f" {(XX.time[0].values.item() ).strftime('%Y-%m-%d')} to {(XX.time[-1].values.item() ).strftime('%Y-%m-%d')}"
dayrange=f"{(XX.time[0].values.item() ).strftime('%Y-%m-%d')}-{(XX.time[-1].values.item() ).strftime('%d')}"
#loot=f"Lat={A.X.lat[jj].values:.1f}"
latval = lat1[jj]
if latval < 0:
    ILat=f'{str(int(-latval))}S'
else:
    ILat=f'{str(int(latval))}N'

print(f" Lat={latval:.1f} ")
print(tootrange )

plev=XX.lev.values
zlev=-7.0*np.log(plev/1_000.)
klist=[0] #np.arange( len(zlev) )
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    if zlev[k]>30. :
        scak=scak*0.25
    #co=ax[p].contourf( scak*ogOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co=ax[p].contourf(lon1, days, scak* varTS_x1[:,jj,:],levels=dolev ,cmap='bwr' )
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    ax[p].set_ylabel( 'Days' )
    ax[p].set_xlabel( 'Longitude' )
    cb=plt.colorbar( co )
    p=p+1

fig.suptitle( f'{exp}, {tootrange}, Lat={latval:.1f}' , fontsize=20)


In [ ]:
print(np.shape(tsO_xQ))

In [ ]:
varTS_XX=devTS_XX*devTS_XX
rmsTS_XX=np.sqrt( np.average( varTS_XX, axis=0) )

In [ ]:
#plt.contourf( devTS_XX[10,:,:] ,levels=np.linspace(-1,1,41) )
plt.contourf( lonO, latO, rmsTS_XX ,levels=np.linspace(-1,1,41) )
plt.colorbar()

In [ ]:
plt.contourf( np.average(varTS_x1,axis=0) ,levels=np.linspace(-1,1,41) )
plt.colorbar()

In [ ]:
plt.contourf( devTS_XX[:,300,:]  , levels=np.linspace(-1,1,41)   )